In [1]:
%load_ext autoreload
%autoreload
from simba import transfer_function_to_graph, tf2rss, adiabatically_eliminate
from sympy import symbols, simplify, Matrix, sqrt, conjugate

First looking at passive realisation of coupled cavity setup with coupling constant $g = 0$

In [ ]:
s = symbols('s')
gamma_f, omega_s = symbols('gamma_f omega_s', real=True, positive=True)
tf = (s**2 + s * gamma_f + omega_s**2) / (s**2 - s * gamma_f + omega_s**2)

transfer_function_to_graph(tf, 'passive_coupled_cavity.png', layout='dot')


![](passive_coupled_cavity.png)

In [ ]:
split_network = tf2rss(tf).to_slh().split()

In [ ]:
h_int = split_network.interaction_hamiltonian
h_int.expr.simplify()

In [ ]:
split_network.interaction_hamiltonian.h

In [ ]:
h_int.states

In [ ]:
simplify(h_int.dynamical_matrix)

In [ ]:
eqns = split_network.frequency_domain_eqns
eqns.eqns

In [ ]:
eqns.states.T

In [ ]:
# Calculating the input-output transfer function
ain1, aout1 = eqns.get_symbols(['ain_1', 'aout_1'])
aout1 = eqns.solve([ain1, conjugate(ain1)])[aout1]
tf = (aout1 / ain1).simplify()
gamma_1, _ = split_network.aux_coupling_constants
adiabatically_eliminate(tf, gamma_1)

In [ ]:
(s**2 + s * gamma_f + omega_s**2) / (s**2 - s * gamma_f + omega_s**2)

Now looking at the active realisation ($g \neq 0$)

In [ ]:
# parameterise with lambda = g**2 - omega_s**2 > 0
lmbda = symbols('lambda', real=True, positive=True)
tf = (s**2 + s * gamma_f - lmbda) / (s**2 - s * gamma_f - lmbda)

transfer_function_to_graph(tf, 'active_coupled_cavity.png', layout='dot')

![](active_coupled_cavity.png)

In [ ]:
split_network = tf2rss(tf).to_slh().split()
h_int = split_network.interaction_hamiltonian
h_int.expr.simplify()

In [ ]:
simplify(h_int.dynamical_matrix)

In [ ]:
eqns = split_network.frequency_domain_eqns
eqns.eqns

In [ ]:
ain1 = eqns.states[8]
aout1 = eqns.solve([8, 9])[10]
tf = (aout1 / ain1).simplify()
gamma_1, _ = split_network.aux_coupling_constants
adiabatically_eliminate(tf, gamma_1)

In [ ]:
(s**2 + s * gamma_f - lmbda) / (s**2 - s * gamma_f - lmbda)

Differs by phase shift of $\pi$

Now let's look at the transfer function from $a_1$ to $aout_1$

In [ ]:
eqns.states.T

In [ ]:
a1, a1d, aout_1 = eqns.get_symbols(['a_1', 'conjugate(a_1)', 'aout_1'])
aout1 = eqns.solve([a1, a1d])[aout_1]
tf = (aout1 / a1).simplify()
gamma_1, _ = split_network.aux_coupling_constants
tf = adiabatically_eliminate(tf, gamma_1)
tf

In [ ]:
from sympy import I, pi, lambdify
params = {gamma_f: 1000, lmbda: 2000}
f = symbols('f')
replaced = tf.subs(params).subs(s, I*2*pi*f).simplify()
fn = lambdify(f, replaced, 'numpy')

import numpy as np
import matplotlib.pyplot as plt

fs = np.logspace(0, 3, num=1000)
ys = fn(fs)

fig, ax = plt.subplots()
ax.loglog(fs, abs(ys))
plt.show()